In [1]:
import os
import pathlib
from pprint import pprint
import json

In [2]:
from dotenv import load_dotenv
load_dotenv()
print(os.getenv('RBAC'))

FALSE


In [3]:
# download spec files from azure storage account
from prompteval.downloadSpecFiles import downloadApiSpecFiles
api_func_folder = pathlib.Path('./apisrc')
downloadApiSpecFiles("apisrc")

Blob cstest.cs downloaded to C:\Users\wenyutang\workspace\eval-prompt\prompteval\..\apisrc\cstest.cs
Blob pet.js downloaded to C:\Users\wenyutang\workspace\eval-prompt\prompteval\..\apisrc\pet.js
Blob pytest.py downloaded to C:\Users\wenyutang\workspace\eval-prompt\prompteval\..\apisrc\pytest.py


In [3]:
# generate yml file from prompt file
# api function file folder
from prompteval.prompteval import get_language_from_extension, generateApiSpec, remove_code_block_format, clean_folder
api_func_folder = pathlib.Path('./apisrc')
res_folder_path = pathlib.Path('./results')
clean_folder(res_folder_path)
for api_file in api_func_folder.glob('*'):
    with open(api_file, 'r', encoding='utf-8') as f:
        api_func = f.read()
    file_extension = api_file.suffix[1:]
    language = get_language_from_extension(file_extension)
    response = generateApiSpec(api_func, language)
    filename = f"{api_file.stem}_response.yml"
    output_file = res_folder_path / filename
    with open(output_file, "w", encoding="utf-8") as out_file:
        out_file.write(remove_code_block_format(response["response"]))

In [ ]:
# spectral analysis for results
from prompteval.evaluate import any_spec_res, text_to_json_objects, spectral_analyse
ruleset_url = "https://raw.githubusercontent.com/Azure/APICenter-Analyzer/preview/resources/rulesets/oas.yaml"
res_folder_path = pathlib.Path('./results')
unique_results = set()
calcs = []
for res_file in res_folder_path.glob("*"):
    if res_file.suffix == ".yml":
      out_file = os.path.basename(res_file.stem + "_output.json")
      spectral_analyse(os.path.basename(res_file), ruleset_url, res_folder_path, os.path.basename(out_file))
      res = any_spec_res(text_to_json_objects(res_folder_path, out_file))
      for item in res:
          unique_results.add(item)

In [ ]:
pprint(unique_results)
with open(pathlib.Path(res_folder_path, 'output.jsonl'), 'w') as json_file:
    for item in unique_results:
        # 将每个对象转换回JSON字符串，并写入文件，每个对象占一行
        json.dump(item.to_json(), json_file)
        json_file.write('\n')